# Support notebook for Basic Protocol 2 
# Select a few samples from a database file

Based on @mhaghigh code

# Import libraries

In [1]:
import pandas as pd
import numpy as np
# from utils.read_data import *
# from utils.visualize_data import *
from functools import reduce
from sklearn.cluster import KMeans
from openpyxl import Workbook, load_workbook
import os
import easygui as eg
from numpy import random
import sqlite3
import pycytominer
import boto3

In [3]:
s3 = boto3.resource('s3')

In [16]:
BUCKET_NAME = 'cellpainting-gallery'
KEY = 'cpg0004-lincs' 

In [4]:
bucket_name = "cellpainting-gallery"
bucket_dir = "cpg0004-lincs/broad/images/2016_04_01_a549_48hr_batch1/images/SQ00014812__2016-05-23T20_44_31-Measurement1/Images/"
filename = 'r01c01f01p01-ch1sk1fk1fl1.tiff'
s3.meta.client.download_file(Filename=filename,Bucket=bucket_name,Key=bucket_dir + filename)

In [21]:
bucket_name = "cellpainting-gallery"
bucket_dir = "cpg0004-lincs/broad/workspace/backend/2016_04_01_a549_48hr_batch1/SQ00015059/"
filename = 'SQ00015059.sqlite'
s3.meta.client.download_file(Filename=filename,Bucket=bucket_name,Key=bucket_dir + filename)

## Import database file

- This specific database file is available on cpg0004-lincs. More details to download see https://github.com/broadinstitute/cellpainting-gallery

In [13]:
from pycytominer.cyto_utils.cells import SingleCells
database = SingleCells(f"sqlite:///G:/My Drive/Analysis/CellRecovery/workspace/backend/220530_150503_Plate_1.sqlite")
df = database.merge_single_cells()

In [14]:
df

,Metadata_ImageNumber,Metadata_Plate,Metadata_Site,Metadata_Well,Metadata_TableNumber,Metadata_ObjectNumber_cytoplasm,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,...,Nuclei_Texture_Variance_Nfkb_10_02_256,Nuclei_Texture_Variance_Nfkb_10_03_256,Nuclei_Texture_Variance_Nfkb_20_00_256,Nuclei_Texture_Variance_Nfkb_20_01_256,Nuclei_Texture_Variance_Nfkb_20_02_256,Nuclei_Texture_Variance_Nfkb_20_03_256,Nuclei_Texture_Variance_Nfkb_5_00_256,Nuclei_Texture_Variance_Nfkb_5_01_256,Nuclei_Texture_Variance_Nfkb_5_02_256,Nuclei_Texture_Variance_Nfkb_5_03_256
0,2,220530_150503_Plate_1,10,B10,2336454233,1,5063,12495,1170,341,...,45.589563,53.325398,66.629519,59.852678,44.582693,85.098378,44.037665,50.674460,53.179678,44.612971
1,2,220530_150503_Plate_1,10,B10,2336454233,2,14205,30686,1106,419,...,17.132255,16.983792,16.784617,17.904774,17.402170,17.707461,16.995621,16.728585,17.002814,16.793547
2,2,220530_150503_Plate_1,10,B10,2336454233,3,10030,15688,546,639,...,38.067857,39.045601,36.387262,68.029233,54.976458,44.386662,39.056549,36.102692,34.377453,35.137296
3,2,220530_150503_Plate_1,10,B10,2336454233,4,8430,14880,508,743,...,38.706969,41.103461,45.870498,36.085891,43.831152,64.030715,36.923310,37.512770,38.399534,36.703622
4,2,220530_150503_Plate_1,10,B10,2336454233,5,17887,30816,363,861,...,11.248281,11.296102,12.636678,14.382164,11.863303,11.992286,11.389719,11.326392,11.522179,11.655348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549,2155,220530_150503_Plate_1,4,G9,2131253378,4,19313,31390,845,333,...,23.080311,23.638556,23.638933,25.370321,22.964257,20.104101,22.443713,22.760637,22.823240,22.888850
8550,2155,220530_150503_Plate_1,4,G9,2131253378,5,6670,12727,947,426,...,1100.724153,1174.726232,1155.369012,1131.086735,1060.402542,1342.505908,1108.094041,1071.288743,1090.060471,1069.632574
8551,2155,220530_150503_Plate_1,4,G9,2131253378,6,22662,43290,973,572,...,27.091926,27.664021,27.039187,31.273461,27.152268,21.399437,27.321818,26.229668,27.286107,29.910903
8552,2155,220530_150503_Plate_1,4,G9,2131253378,7,20956,35397,1007,702,...,9.267765,9.502285,10.339168,9.094232,9.065310,11.876419,9.037987,9.410834,9.497060,9.100095


In [52]:
from pycytominer.cyto_utils.cells import SingleCells
database = SingleCells(f"https://cellpainting-gallery.s3.us-east-1.amazonaws.com/cpg0004-lincs/broad/workspace/backend/2016_04_01_a549_48hr_batch1/SQ00015059/SQ00015059.sqlite")
df = database.merge_single_cells()

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:https

In [10]:
conn = sqlite3.connect(r"G:\My Drive\Analysis\CellRecovery\workspace\backend\220530_150503_Plate_1.sqlite")

## Define which wells to retrieve from the dataset

In [4]:
well_name = ["A02", "A07", "B07", "D19", "D21", "H13", "J19", "L19"]

channels = ["DNA","ER","RNA","AGP","Mito"]

## Retrieve image numbers from well's names

In [5]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])

Image
Cells
Cytoplasm
Nuclei


In [6]:
#get the names of each well as a list from well_name
rand_img_num=well_name.copy()

list_str="('"
for i in rand_img_num:
    list_str=list_str+str(i)+"','" 
list_str=list_str[:-2]+")"

print(list_str)

('A02','A07','B07','D19','D21','H13','J19','L19')


In [7]:
#define the name of the well
meta_well_col_str="Image_Metadata_Well"

In [8]:
#create a query with the compartment names to be able to retrieve them from the table
compartment_query = "select * from {} WHERE {} IN {};".\
format("Image",meta_well_col_str,list_str)

In [9]:
#select the metadata and image measurements from the Image table belonging to the wells you want
plateImageDf = pd.read_sql(sql=compartment_query, con=conn)

In [10]:
#get the ImageNumbers for the wells you want
img_nums=plateImageDf.ImageNumber.unique().tolist()

list_str2="("
for i in img_nums:
    list_str2=list_str2+str(i)+',' 
list_str2=list_str2[:-1]+")"

print(list_str2)

(10,11,12,13,14,15,16,17,18,55,56,57,58,59,60,61,62,63,271,272,273,274,275,276,277,278,279,810,811,812,813,814,815,816,817,818,828,829,830,831,832,833,834,835,836,1620,1621,1622,1623,1624,1625,1626,1627,1628,2106,2107,2108,2109,2110,2111,2112,2113,2114,2538,2539,2540,2541,2542,2543,2544,2545,2546)


In [11]:
#from the compartment names, open each table and retrieve information from the images on list_str2
plateDf_list=[]
compartments=["cells", "cytoplasm", "nuclei"]

for compartment in compartments:
    compartment_query = "select * from {} WHERE {} IN {};".format(compartment,"ImageNumber",list_str2)
    plateDf_list.append(pd.read_sql(sql=compartment_query, con=conn))

In [14]:
#create the dataframe from the Df_list 
plateDf = reduce(lambda left,right: pd.merge(left,right,on=["TableNumber", "ImageNumber", "ObjectNumber"]), plateDf_list)

In [15]:
#merge information from Image table with cells data
plateDfwMeta = pd.merge(plateDf, plateImageDf, on=["TableNumber", "ImageNumber"])

In [16]:
#exclude duplicated cols
plateDfwMeta = plateDfwMeta.loc[:,~plateDfwMeta.columns.duplicated()]

In [17]:
#create a copy
df_p_s0 = plateDfwMeta

In [18]:
#take a look at the dataframe
print(df_p_s0.shape)
df_p_s0.head()

(7577, 2442)


,TableNumber,ImageNumber,ObjectNumber,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Image_Width_IllumAGP,Image_Width_IllumDNA,Image_Width_IllumER,Image_Width_IllumMito,Image_Width_IllumRNA,Image_Width_OrigAGP,Image_Width_OrigDNA,Image_Width_OrigER,Image_Width_OrigMito,Image_Width_OrigRNA
0,1240fc906b760746bfaa4913fdb947b6,10,1,7334,642.0,0.0,1.196197,0.732293,1.0,0.622053,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
1,1240fc906b760746bfaa4913fdb947b6,10,2,7084,1952.0,1.0,1.288208,0.865021,1.0,0.572722,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
2,1240fc906b760746bfaa4913fdb947b6,10,3,13696,2159.0,40.0,1.116869,0.632134,1.0,0.689766,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
3,1240fc906b760746bfaa4913fdb947b6,10,4,9814,1123.0,36.0,1.147676,0.773804,1.0,0.702606,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
4,1240fc906b760746bfaa4913fdb947b6,10,5,16297,1672.0,78.0,1.071092,0.708822,1.0,0.691606,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160


## Save selected DataFrame to csv

In [19]:
import pickle
df_p_s0.to_csv("SQ00015195_raw.csv")